In [2]:
# Import the necessary modules to demonstrate the validator
import bittensor as bt
from argparse import ArgumentParser
import requests
import image_generation_subnet as ig_subnet
from neurons.validator.validator import Validator

In [11]:
# Initialize the validator

parser = ArgumentParser()
parser.add_argument("--netuid", type=int, default=23, help="The chain subnet uid.")

# Define the bittensor configuration parameters here
parser.add_argument("--wallet.name", type=str, default="default")
parser.add_argument("--wallet.hotkey", type=str, default="default")
parser.add_argument("--subtensor.network", type=str, default="finney")
parser.add_argument("--neuron.axon_off", type=bool, default=True)

config = bt.config( parser )
print(config)

metagraph = bt.metagraph(netuid=config.netuid)

nicheimage_validator: Validator = Validator(config)


netuid: 23
wallet:
  name: default
  hotkey: default
subtensor:
  network: finney
neuron:
  axon_off: true
config: null
strict: false
no_version_checking: false
no_prompt: false

2024-04-18 09:18:19.969 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-04-18 09:18:19.969 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-04-18 09:18:19.969 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-04-18 09:18:20.717 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.


In [12]:
def get_top_miner(metagraph) -> bt.NeuronInfo:
    from torch import argmax
    # Get incentives and the top performing UID
    incentives = metagraph.I
    stakes = metagraph.S
    incentives[stakes > 10000] = 0
    uid = argmax(incentives).item()
    print(f"Top UID: {uid} with incentive: {incentives[uid]}")

    # Return AxonInfo based on the UID
    neuron_info = metagraph.neurons[uid]
    return neuron_info

def prepare_challenge(validator: Validator, miner_uid):
    import random
    model_name = validator.miner_manager.all_uids_info[miner_uid]["model_name"]
    print(f"Model name: {model_name}")
    pipeline_type = random.choice(
        validator.nicheimage_catalogue[model_name]["supporting_pipelines"]
    )
    print(f"Pipeline type: {pipeline_type}")
    synapses, batched_uids_should_rewards = validator.prepare_challenge(uids_should_rewards=[(miner_uid, True)], pipeline_type=pipeline_type, model_name=model_name)
    synapse = synapses[0]
    uid, should_reward = batched_uids_should_rewards[0]
    print(f"Synapse: {synapse}")
    print(f"UID: {uid}")
    print(f"Should reward: {should_reward}")
    return synapse, uid

def start_query(validator: Validator, synapse, uid):
    model_name = validator.miner_manager.all_uids_info[uid]["model_name"]
    dendrite = bt.dendrite(validator.wallet)
    base_synapse = synapse.copy()
    axons = [validator.metagraph.axons[int(uid)]]
    responses = dendrite.query(
        axons=axons,
        synapse=synapse,
        deserialize=False,
        timeout=validator.nicheimage_catalogue[model_name]["timeout"],
    )
    response = responses[0]

    # Calculate the reward

    reward_url = validator.nicheimage_catalogue[model_name]["reward_url"]

    if callable(reward_url):
        uids, rewards = reward_url(
            base_synapse, [response], [uid]
        )
    else:
        uids, rewards = ig_subnet.validator.get_reward(
            reward_url,
            base_synapse,
            [response],
            [uid],
            validator.nicheimage_catalogue[model_name].get("timeout", 12),
        )

    print(f"UIDs: {uids}")
    print(f"Rewards: {rewards}")

    print(f"Final response: {response}, is successful: {response.is_success}")
    return response.deserialize()



Top UID: 127 with incentive: 0.015579461120069027


NeuronInfoLite(hotkey='5DCnissVuUorgYZaBJyBdarPxUQnLypfwYmoMW5TAKm5GRvE', coldkey='5DiEASKwwEvUDZLZN1JSwEJkJPaDRqFNpYdCcsqFwVVMZo4b', uid=127, netuid=23, active=False, stake=τ1.727654778, stake_dict={'5DiEASKwwEvUDZLZN1JSwEJkJPaDRqFNpYdCcsqFwVVMZo4b': τ1.727654778}, total_stake=τ1.727654778, rank=0.015579461356527047, emission=0.035376446, incentive=0.015579461356527047, consensus=0.016861219195849545, trust=0.9846494239719235, validator_trust=0.0, dividends=0.0, last_update=2496667, validator_permit=False, prometheus_info=PrometheusInfo(block=0, version=0, ip='0.0.0.0', port=0, ip_type=0), axon_info=AxonInfo( /ipv4/74.226.197.139:10004, 5DCnissVuUorgYZaBJyBdarPxUQnLypfwYmoMW5TAKm5GRvE, 5DiEASKwwEvUDZLZN1JSwEJkJPaDRqFNpYdCcsqFwVVMZo4b, 693 ), pruning_score=4225, is_null=False)

### Query top miner step by step

In [ ]:
# Get the top miner
top_miner = get_top_miner(metagraph)

# Prepare the challenge
synapse, uid = prepare_challenge(nicheimage_validator, top_miner.uid)

# Start the query
response = start_query(nicheimage_validator, synapse, uid)
print(response)

### Explaining rewarding mechanism
- **Incentivized volume rewarding mechanism**: The reward formula incorporates the volume of requests handled, calculated as follows:
  - `new_reward = (matching_result - time_penalty) * (0.8 + 0.2 * volume_scale)`
  - `matching_result` is 0 or 1 based on the similarity matching result with the reproduction of the validator.
  - `time_penalty = 0.4 * (processing_time / timeout)**3`
  - `volume_scale = max(min(total_volume**0.5 / 1000**0.5, 1), 0)`


In [ ]:

def add_time_penalty(rewards, process_times, max_penalty=0.4, factor: float = 12):
    """
    Add time penalty to rewards, based on process time
    """
    penalties = [
        max_penalty * pow(process_time, 3) / pow(factor, 3)
        for process_time in process_times
    ]
    penalties = [min(penalty, max_penalty) for penalty in penalties]
    for i in range(len(rewards)):
        if rewards[i] > 0:
            rewards[i] = rewards[i] - penalties[i]
    return rewards


def get_reward(
    url: str,
    base_synapse: ig_subnet.protocol.ImageGenerating,
    synapses: list[ig_subnet.protocol.ImageGenerating],
    uids: list[int],
    timeout: float,
) -> list[float]:
    valid_uids = [uid for uid, response in zip(uids, synapses) if response.is_success]
    invalid_uids = [
        uid for uid, synapse in zip(uids, synapses) if not synapse.is_success
    ]
    total_uids = valid_uids + invalid_uids
    valid_synapses = [synapse for synapse in synapses if synapse.is_success]
    if valid_uids:
        data = {
            "miner_data": [synapse.deserialize() for synapse in valid_synapses],
            "base_data": base_synapse.deserialize_input(),
        }
        response = requests.post(url, json=data)
        if response.status_code != 200:
            raise Exception(f"Error in get_reward: {response.json()}")
        valid_rewards = response.json()["rewards"]
        valid_rewards = [float(reward) for reward in valid_rewards]
        process_times = [synapse.dendrite.process_time for synapse in valid_synapses]
        if timeout > 12:
            valid_rewards = add_time_penalty(valid_rewards, process_times, 0.4, 64)
        else:
            valid_rewards = add_time_penalty(valid_rewards, process_times, 0.4, 12)
        valid_rewards = [round(num, 3) for num in valid_rewards]
    else:
        bt.logging.info("0 valid responses in a batch")
        valid_rewards = []

    total_rewards = valid_rewards + [0] * len(invalid_uids)

    return total_uids, total_rewards